In [0]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
!pip install pytorch-transformers

     |████████████████████████████████| 184kB 8.3MB/s 
     |████████████████████████████████| 675kB 16.8MB/s 
     |████████████████████████████████| 860kB 22.8MB/s 
     |████████████████████████████████| 1.0MB 46.9MB/s 
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609184 sha256=7f8b273335f74652a3b8dc60999bf590121f0cc1516957ec25b892c5e809ec55
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=a08b46e55b86eb25a8be35b626b1e244d17505b8a4494e2c43d630faef649d57
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built regex sacremoses


In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_transformers import *
#from pytorch_transformers import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

Using TensorFlow backend.


In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [0]:
import pandas as pd

In [0]:
datafile = pd.read_csv("subtaskA_data_all.csv")

In [0]:
datafile.head()

,id,sent0,sent1
0,0,He poured orange juice on his cereal.,He poured milk on his cereal.
1,1,He drinks apple.,He drinks milk.
2,2,Jeff ran a mile today,"Jeff ran 100,000 miles today"
3,3,A mosquito stings me,I sting a mosquito
4,4,A niece is a person.,A giraffe is a person.


In [0]:
sample_to_disregard = []
for i in range(len(datafile)):
  s0 = datafile[datafile['id'] == i].sent0.values[0]
  s1 = datafile[datafile['id'] == i].sent1.values[0]
  if s0 == s1:
    print("this sample is wrong: ", i)
    sample_to_disregard.append(i)

this sample is wrong:  4075
this sample is wrong:  4121
this sample is wrong:  4313
this sample is wrong:  6398
this sample is wrong:  7700


In [0]:
labelfile = pd.read_csv("subtaskA_answers_all.csv",header = None)

In [0]:
labelfile.head()

,0,1
0,0,0
1,1,0
2,2,1
3,3,1
4,4,1


In [0]:
sample_to_disregard

[4075, 4121, 4313, 6398, 7700]

In [0]:
clean_datafile = datafile.drop(sample_to_disregard,axis=0)

In [0]:
clean_datafile.loc[7699:7701]

,id,sent0,sent1
7699,7699,"It suddenly cooled down, so he caught a cold","It suddenly warmed up, so he caught a cold"
7701,7701,He didn't eat for a month,He hasn't eaten all day


In [0]:
clean_labelfile = labelfile.drop(sample_to_disregard,axis=0)
clean_labelfile.loc[7699:7701]

,0,1
7699,7699,1
7701,7701,0


In [0]:
traindata = clean_datafile[2000:]
trainlabel = clean_labelfile[2000:]

In [0]:
traindata.shape

(7995, 3)

In [0]:
testdata = clean_datafile[0:2000]
testlabel = clean_labelfile[0:2000]

In [0]:
testlabel.shape

(2000, 2)

In [0]:
trainlabel.iloc[0]

0    0
1    0
Name: 0, dtype: int64

In [0]:
sentences0 = traindata.sent0.values
sentences1 = traindata.sent1.values

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 903699.28B/s]


In [0]:
tokenized_s0 = [tokenizer.tokenize(sent) for sent in sentences0]

In [0]:
for i in range(len(tokenized_s0)):
  #sent = tokenized_s0[i]
  if tokenized_s0[i][-1] == '.':
    continue
  else:
    tokenized_s0[i].append('.')


In [0]:
tokenized_s1 = [tokenizer.tokenize(sent) for sent in sentences1]

In [0]:
for i in range(len(tokenized_s1)):
  #sent = tokenized_s0[i]
  if tokenized_s1[i][-1] == '.':
    continue
  else:
    tokenized_s1[i].append('.')


In [0]:
tokenized_s1[1]

['she', 'wore', 'her', 'car', '.']

In [0]:
for i in range(len(tokenized_s0)):
  tokenized_s0[i].insert(0,"[CLS]")
  tokenized_s0[i].append("[SEP]")

In [0]:
tokenized_s0[1]

['[CLS]', 'she', 'wore', 'her', 'coat', '.', '[SEP]']

In [0]:
for i in range(len(tokenized_s1)):
  #tokenized_s0[i].insert(0,"[CLS]")
  tokenized_s1[i].append("[SEP]")

In [0]:
tokenized_s1[1]

['she', 'wore', 'her', 'car', '.', '[SEP]']

In [0]:
tokenized_texts = []
for i in range(len(tokenized_s1)):
  #tokenized_s0[i].append(tokenized_s1[i])
  tokenized_texts.append(tokenized_s0[i]+tokenized_s1[i])

In [0]:
tokenized_texts[1]

['[CLS]',
 'she',
 'wore',
 'her',
 'coat',
 '.',
 '[SEP]',
 'she',
 'wore',
 'her',
 'car',
 '.',
 '[SEP]']

In [0]:
# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
#sentences = ["[CLS] " + s0 + " [SEP]"  for s0 in sentences0 ]
#for i in range(len(sentences)):
  #sentences[i] = sentences[i] + " " + sentences1[i] + " [SEP]"

In [0]:
#sentences[0]

In [0]:
labels = trainlabel[1].values

In [0]:
labels

array([1, 1, 0, ..., 1, 1, 0])

In [0]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

#tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
#print ("Tokenize the first sentence:")
#print (tokenized_texts[0])

In [0]:
MAX_LEN = 128

In [0]:
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [0]:
input_ids[0]

array([  101,  2336,  4553,  8785,  1999, 12463,  1012,   102,  2336,
        4521,  8785,  1999, 12463,  1012,   102,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0]

In [0]:
# segement id
segment_ids = []
for i in range(len(traindata)):
  segment_ids.append([0] * len(tokenized_s0[i]) + [1]*len(tokenized_s1[i]))
segment_ids = pad_sequences(segment_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [0]:
segment_ids[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [0]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [0]:
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)


train_segments, validation_segments,_,_ = train_test_split(segment_ids, input_ids,
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [0]:
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_segments = torch.tensor(train_segments)
validation_segments = torch.tensor(validation_segments)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [0]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_segments,train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_segments,validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [0]:
config = BertConfig.from_pretrained("bert-base-uncased", output_hidden_states=True)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", config=config)
#model = BertForSequenceClassification.from_pretrained("bert-base-uncased",output_hidden_states=True)
model.cuda()

100%|██████████| 440473133/440473133 [00:16<00:00, 26505253.58B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [0]:
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=100, t_total=1000) 

In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4
# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_segment_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss, logits_train, hidden_layers_train = model(b_input_ids, b_segment_ids, attention_mask=b_input_mask, labels=b_labels)
    #loss, logits_train, hidden_layers_train = outputs
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    scheduler.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_segment_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits, hidden_layers_val = model(b_input_ids, b_segment_ids, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))  

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 0.5942636720339457


Epoch:  25%|██▌       | 1/4 [01:36<04:48, 96.15s/it]

Validation Accuracy: 0.7825
Train loss: 0.31040756791830065


Epoch:  50%|█████     | 2/4 [03:12<03:12, 96.13s/it]

Validation Accuracy: 0.83875
Train loss: 0.1346483389991853


Epoch:  75%|███████▌  | 3/4 [04:48<01:36, 96.10s/it]

Validation Accuracy: 0.85125
Train loss: 0.06545066107064486


Epoch: 100%|██████████| 4/4 [06:24<00:00, 96.07s/it]

Validation Accuracy: 0.845


leave out test data result

In [0]:
sentences0 = testdata.sent0.values
sentences1 = testdata.sent1.values

In [0]:
tokenized_s0 = [tokenizer.tokenize(sent) for sent in sentences0]

In [0]:
for i in range(len(tokenized_s0)):
  #sent = tokenized_s0[i]
  if tokenized_s0[i][-1] == '.':
    continue
  else:
    tokenized_s0[i].append('.')


In [0]:
tokenized_s0[0]

['he', 'poured', 'orange', 'juice', 'on', 'his', 'cereal', '.']

In [0]:
tokenized_s1 = [tokenizer.tokenize(sent) for sent in sentences1]

In [0]:
for i in range(len(tokenized_s1)):
  #sent = tokenized_s0[i]
  if tokenized_s1[i][-1] == '.':
    continue
  else:
    tokenized_s1[i].append('.')


In [0]:
tokenized_s1[0]

['he', 'poured', 'milk', 'on', 'his', 'cereal', '.']

In [0]:
for i in range(len(tokenized_s0)):
  tokenized_s0[i].insert(0,"[CLS]")
  tokenized_s0[i].append("[SEP]")

In [0]:
tokenized_s0[0]

['[CLS]',
 'he',
 'poured',
 'orange',
 'juice',
 'on',
 'his',
 'cereal',
 '.',
 '[SEP]']

In [0]:
for i in range(len(tokenized_s1)):
  #tokenized_s0[i].insert(0,"[CLS]")
  tokenized_s1[i].append("[SEP]")

In [0]:
tokenized_s1[0]

['he', 'poured', 'milk', 'on', 'his', 'cereal', '.', '[SEP]']

In [0]:
tokenized_texts = []
for i in range(len(tokenized_s1)):
  #tokenized_s0[i].append(tokenized_s1[i])
  tokenized_texts.append(tokenized_s0[i]+tokenized_s1[i])

In [0]:
tokenized_texts[0]

['[CLS]',
 'he',
 'poured',
 'orange',
 'juice',
 'on',
 'his',
 'cereal',
 '.',
 '[SEP]',
 'he',
 'poured',
 'milk',
 'on',
 'his',
 'cereal',
 '.',
 '[SEP]']

In [0]:
# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
#sentences = ["[CLS] " + s0 + " [SEP]"  for s0 in sentences0 ]
#for i in range(len(sentences)):
  #sentences[i] = sentences[i] + " " + sentences1[i] + " [SEP]"

In [0]:
#sentences[0]

In [0]:
labels = testlabel[1].values

In [0]:
labels

array([0, 0, 1, ..., 0, 0, 1])

In [0]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

#tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
#print ("Tokenize the first sentence:")
#print (tokenized_texts[0])

In [0]:
MAX_LEN = 128
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [0]:
len(input_ids)

2000

In [0]:
# segement id
segment_ids = []
for i in range(len(testdata)):
  segment_ids.append([0] * len(tokenized_s0[i]) + [1]*len(tokenized_s1[i]))
segment_ids = pad_sequences(segment_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [0]:
len(segment_ids)

2000

In [0]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [0]:
batch_size = 16  
test_inputs = input_ids
test_segments = segment_ids
test_labels = labels
test_masks = attention_masks


test_inputs = torch.tensor(test_inputs)
test_segments = torch.tensor(test_segments)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_masks)

test_data = TensorDataset(test_inputs, test_segments,test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [0]:
model.eval()

# Tracking variables 
predictions , true_labels = [], []
input_id_list = []
#last_layer = []


# Predict 
for batch in test_dataloader:
  #print(batch)
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_segment_ids, b_input_mask, b_labels = batch
  input_id_list.append(b_input_ids.detach().cpu())
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits, hiddens = model(b_input_ids, b_segment_ids, attention_mask=b_input_mask)
  
  #print("hidden_size: ", len(hiddens))

  # Move logits and labels to CPU
  #print("logits_before_detached: ", logits)
  logits = logits.detach().cpu().numpy()

  #print("logits_detached: ", logits)
  
  #print("hiddens: ", hiddens)
  #temp_hidden = []
  #for i in range(len(hiddens)):
  #temp_hiddeni = list(hiddens)[11].detach().cpu().numpy()
    #temp_hidden.append(temp_hiddeni)
  #print("tempi size ",len(temp_hiddeni))
  
  #last_layer.append(temp_hiddeni)
    #print(i,)
    #hiddens[i] = hiddens[i].detach().cpu().numpy()
  #hiddens = hiddens.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)
  #hidden_layers.append(hiddens)


In [0]:
test_accuracy = []

for i in range(len(true_labels)):
  acc = flat_accuracy(predictions[i], true_labels[i])
  test_accuracy.append(acc)

In [0]:
len(test_accuracy)

63

In [0]:
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

In [0]:
np.sum(flat_predictions == flat_true_labels) / len(flat_true_labels)

0.9125

In [0]:
flat_ids = [item.numpy() for sublist in input_id_list for item in sublist]

In [0]:
flat_ids[233] == input_ids[233]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]

In [0]:
np.sum(flat_predictions ==labels) / len(flat_true_labels)

0.9125

In [0]:
testlabel

,0,1
0,0,0
1,1,0
2,2,1
3,3,1
4,4,1
...,...,...
1995,1995,0
1996,1996,0
1997,1997,0
1998,1998,0


In [0]:
flat_predictions

array([1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,

In [0]:
flat_true_labels

[0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,


In [0]:
flat_true_labels == labels

array([ True,  True,  True, ...,  True,  True,  True])

In [0]:
result_dict = {}

In [0]:
for i in range(len(flat_true_labels)):
  result_dict[i] = flat_predictions[i]

In [0]:
len(result_dict)

2000

In [0]:
import pickle

In [0]:
with open("pair_classification_predction_0_2000.pickle","wb") as handle:
  pickle.dump(result_dict,handle)

In [0]:
result_backup = [1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0]